In [114]:
import pandas as pd
import numpy as np
import librosa
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation ,Dense ,Dropout,Flatten
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [7]:
file = "F:/Git_projects/audio_classification/UrbanSound8K/17578-5-0-2.wav"
metadata = pd.read_csv("F:/Git_projects/audio_classification/UrbanSound8K/metadata/UrbanSound8K.csv")
file_path = "F:\Git_projects/audio_classification/UrbanSound8K/audio"

In [8]:
def feature_extractor(file):
    audio_data,sample_rate = librosa.load(file, res_type = 'kaiser_fast')
    temp = librosa.feature.mfcc( y =audio_data, sr = sample_rate, n_mfcc = 40)
    mfcc_scaled = np.mean(temp.T,axis = 0)
    return mfcc_scaled

In [9]:
metadata

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn


In [12]:
features = []
for index, data in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(file_path),"fold" + str(data["fold"]) + "/",str(data["slice_file_name"]))
    mfcc_scaled = feature_extractor(file_name)
    target = data["class"]
    features.append([mfcc_scaled,target])
    

3555it [06:38, 10.81it/s]C:\Users\Lenovo\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1323
  warnings.warn(
8326it [15:15, 14.02it/s]C:\Users\Lenovo\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1103
  warnings.warn(
C:\Users\Lenovo\anaconda3\lib\site-packages\librosa\core\spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=1523
  warnings.warn(
8732it [15:52,  9.17it/s]


In [13]:
features_df = pd.DataFrame(features, columns = ["features","target"])

In [52]:
X = np.array(features_df["features"].to_list())
y = np.array(features_df["target"].to_list())

In [53]:
enc = LabelEncoder()

'children_playing'

In [54]:
target_classes = enc.fit_transform(y)

In [57]:
y = np.array(pd.get_dummies(features_df["target"]))

In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify = y,random_state = 666,test_size = 0.20)

In [99]:
X.shape

(8732, 40)

In [121]:
model = Sequential()

In [122]:
model.add(Dense(100,input_shape = (40,)))
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(300))
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dropout(0.3))

model.add(Dense(10))
model.add(Activation("softmax"))



In [123]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 100)               4100      
_________________________________________________________________
activation_48 (Activation)   (None, 100)               0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 300)               30300     
_________________________________________________________________
activation_49 (Activation)   (None, 300)               0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_54 (Dense)             (None, 100)              

In [124]:
model.compile(loss = "categorical_crossentropy",metrics = ["accuracy"],optimizer = "adam")

In [125]:
model.fit(X_train,y_train,batch_size = 64,epochs = 100,validation_data = (X_test,y_test))

Epoch 1/100
110/110 [==============================] - 1s 5ms/step - loss: 11.8922 - accuracy: 0.1087 - val_loss: 2.2704 - val_accuracy: 0.1385
Epoch 2/100
110/110 [==============================] - 0s 4ms/step - loss: 2.3006 - accuracy: 0.1483 - val_loss: 2.2198 - val_accuracy: 0.1608
Epoch 3/100
110/110 [==============================] - 0s 4ms/step - loss: 2.2334 - accuracy: 0.1624 - val_loss: 2.1215 - val_accuracy: 0.2078
Epoch 4/100
110/110 [==============================] - 0s 4ms/step - loss: 2.1627 - accuracy: 0.1788 - val_loss: 2.0699 - val_accuracy: 0.2261
Epoch 5/100
110/110 [==============================] - 0s 4ms/step - loss: 2.1527 - accuracy: 0.1858 - val_loss: 2.0639 - val_accuracy: 0.2295
Epoch 6/100
110/110 [==============================] - 0s 4ms/step - loss: 2.1115 - accuracy: 0.2124 - val_loss: 2.0254 - val_accuracy: 0.2410
Epoch 7/100
110/110 [==============================] - 0s 4ms/step - loss: 2.0671 - accuracy: 0.2297 - val_loss: 1.9632 - val_accuracy: 0.267

In [71]:
test_path = "F:/Git_projects/audio_classification/UrbanSound8K/344-3-4-0.wav"
temp =feature_extractor(test_path)
temp = temp.reshape(1,-1)

In [143]:
enc.inverse_transform(model.predict_classes(temp))

array(['dog_bark'], dtype='<U16')